In [1]:
import pandas as pd
import subprocess

In [2]:
def get_the_table_data():
    while True:
        table_path = input("What is the table path + name? (.../filename.csv):")
        try:
            video_schnitt_df = pd.read_csv(table_path)
            print("Table loaded succesfully")
            return video_schnitt_df
        except Exception as error:
            print("An error occured:", error)
            continue
        break

In [3]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    print("Table Columns cleaned")
    return df

In [4]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [5]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [6]:
def cut_in_pieces(input_file, timestamp, head_output, tail_output):
    head_cmd = ['ffmpeg', '-i', input_file, '-to', str(timestamp), '-c', 'copy', head_output]
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', str(timestamp), '-c', 'copy', tail_output]

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [7]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'copy', output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [8]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'copy',
               '-c:a', 'aac', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [9]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy',
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [10]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    
    #Construct command
    command = ['ffmpeg',
               '-i', input_file,
               '-ss', cut_head,
               '-to', cut_tail,
               '-vn','-c:a', 'copy', output_file]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get the file and clean it

In [11]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)

Table loaded succesfully
Table Columns cleaned


# Cut head and tail

In [12]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_ohne_start_ende.mp4"
        cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
        cut_tail = video_schnitt_df["hinten_abschneiden_ab"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan":
            cut_head = "00:00:00"
        if str(cut_tail) == "nan":
            cut_tail = "00:50:00"

        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: 'float' object has no attribute 'split'


frame= 9930 fps=0.0 q=-1.0 Lsize=   37699kB time=00:05:30.99 bitrate= 933.0kbits/s speed= 392x    
video:27180kB audio:10238kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.752311%


# Get Standbild

In [13]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_standbild.mp4"
        cut_head = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan":
            continue
        
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: 'float' object has no attribute 'split'


frame=    1 fps=0.0 q=-1.0 Lsize=      73kB time=00:00:00.00 bitrate=N/A speed=   0x    
video:72kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.145466%


# Get audio

In [14]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
    
        # timedelta = cut_tail - pd.to_timedelta(cut_head)
        # dummy_date = pd.Timestamp('1900-01-01')
        # new_timestamp = dummy_date + timedelta
        # cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

        #Define arguments
        output_file = f"{video_name.split(".")[0]}_head_audio.mp4"
        cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
        cut_tail = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

        if cut_head == "nan":
            cut_head = "00:00:00"

        get_audio(video_name,output_file,cut_head,cut_tail)

except Exception as error:
    print("An error occurred:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: 'float' object has no attribute 'split'


size=    5508kB time=00:02:56.98 bitrate= 254.9kbits/s speed= 319x    
video:0kB audio:5475kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.608615%


# Merge audio and video

In [15]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx] == "nan":
            continue
        video_file = f"{video_name.split(".")[0]}_standbild.mp4"
        audio_file = f"{video_name.split(".")[0]}_head_audio.mp4"
        output_file = f"{video_name.split(".")[0]}_merged_start.mp4"
        
        merge_audio_and_video(video_file,audio_file,output_file)

except Exception as error:
    print("An error occured:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: 'float' object has no attribute 'split'


frame=    1 fps=0.1 q=-1.0 Lsize=    1633kB time=00:02:56.98 bitrate=  75.6kbits/s speed=24.1x    
video:72kB audio:1527kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.116739%
[aac @ 0x7fc89b009f00] Qavg: 251.165


# Concatenate videos to end product

In [19]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split(".")[0]}_merged_start.mp4"
        video2 = video_name
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split(".")[0]}_endprodukt.mp4"
        textfile_name = "dummy.txt"

        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

video1: Video_merged_start.mp4 video2 Video.mp4 content: file 'Video_merged_start.mp4'
file 'Video.mp4'


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video1: Median_merged_start.mp4 video2 Median.mp4 content: file 'Median_merged_start.mp4'
file 'Median.mp4'


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video1: Median2_merged_start.mp4 video2 Median2.mp4 content: file 'Median2_merged_start.mp4'
file 'Median2.mp4'


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fd8c001c240] Auto-inserting h264_mp4toannexb bitstream filter


An error occured: 'float' object has no attribute 'split'


frame=22033 fps=11560 q=-1.0 Lsize=   80748kB time=00:15:11.40 bitrate= 725.8kbits/s speed= 478x    
video:55830kB audio:24241kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.845330%


# Cut in pieces

In [ ]:
for idx, video_name in video_schnitt_df["dateiname"].items():
    cut_time = video_schnitt_df["schnitt_setzen_bei"][idx]
    head_output = f"{video_name}_head_piece.mp4"
    tail_output = f"{video_name}_tail_piece.mp4"
    cut_in_pieces(video_name,cut_time,head_output,tail_output)

In [ ]:
# videos need to be in same directory with python script
for idx, video_name in video_schnitt_df["dateiname"].items():
    output_file = f"{video_name}_standbild.mp4"
    cut_time = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

In [ ]:

# Your time string
time_str = "00:00:00.000"

# Convert to Pandas Timedelta
time_delta = pd.to_timedelta(time_str)

# Add one millisecond
new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

# Convert Timedelta to Timestamp for formatting (using a dummy date)
dummy_date = pd.Timestamp('1900-01-01')
new_timestamp = dummy_date + new_time_delta

# Format to "HH:MM:SS.fff"
new_time_str = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

new_time_str


In [ ]:
video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][0]


In [ ]:
duration = pd.to_datetime(get_video_duration("Video.mp4")).strftime('%H:%M:%S.%f')[:-3]
duration = duration + duration

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]
result = start_time + pd.to_timedelta(end_time) 

In [ ]:
get_video_duration("Video.mp4")